In [19]:
from pyquaternion import Quaternion
from nuscenes.nuscenes import NuScenes
from nuscenes.utils.data_classes import RadarPointCloud, Box
from nuscenes.utils.geometry_utils import points_in_box, view_points
from nuscenes.scripts.export_2d_annotations_as_json import post_process_coords
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
import io
import base64
from shapely.geometry import box as shapely_box, MultiPoint


In [45]:
def replace_categories(dataset, replacements):
    """
    Replace specified categories with new categories in the dataset.
    """
    for data in dataset:
        for annotation in data['annotations']:
            if annotation['category_name'] in replacements:
                annotation['category_name'] = replacements[annotation['category_name']]
    return dataset

# Define the replacements for specific categories
category_replacements = {'human.pedestrian.construction_worker': 'pedestrian',
                         'human.pedestrian.adult': 'pedestrian',
                         'human.pedestrian.stroller': 'pedestrian',
                         'human.pedestrian.police_officer': 'pedestrian',
                         'human.pedestrian.personal_mobility': 'pedestrian',
                         'human.pedestrian.wheelchair': 'pedestrian',
                         'vehicle.bus.bendy': 'bus',
                         'human.pedestrian.child':'pedestrian',
                         'vehicle.truck': 'truck',
                         'vehicle.car': 'car',
                         'vehicle.motorcycle':'motorcycle',
                         'vehicle.trailer' : 'trailer',
                         'vehicle.bicycle': 'bicycle',
                         'movable_object.barrier':'barrier',
                         'vehicle.bus.rigid':'bus',
                         'vehicle.emergency.police':'car'
                         }

# Initialize nuScenes dataset
nusc = NuScenes(version='v1.0-trainval', dataroot='', verbose=True)

# Define a function to create your custom dataset with relevant information
def create_custom_dataset(nusc, C):
    dataset = []

    remove_categories = ['static_object.bicycle_rack','vehicle.emergency.ambulance',
                         'movable_object.trafficcone', 'movable_object.debris',
                         'movable_object.pushable_pullable','vehicle.construction','animal']

    def image_to_base64(image_path):
        with open(image_path, "rb") as img_file:
            img_bytes = img_file.read()
            encoded_image = base64.b64encode(img_bytes).decode('utf-8')
        return encoded_image

    def convert_3d_to_2d_bbox(box: Box, camera_intrinsic):
        """
        Convert a 3D bounding box to a 2D bounding box.
        :param box: The 3D bounding box object.
        :return: Tuple representing the minimum and maximum coordinates of the 2D bounding box, or None if no intersection.
        """
        # Translate and rotate the box to the ego-pose frame.
        box.translate(-np.array(pose_rec['translation']))
        box.rotate(Quaternion(pose_rec['rotation']).inverse)
    
        # Translate and rotate the box to the calibrated sensor frame.
        box.translate(-np.array(calibrated_sensor['translation']))
        box.rotate(Quaternion(calibrated_sensor['rotation']).inverse)
    
        # Filter out corners not in front of the calibrated sensor.
        corners_3d = box.corners()
        in_front = np.argwhere(corners_3d[2, :] > 0).flatten()
        corners_3d = corners_3d[:, in_front]
    
        # Project 3D box to 2D.
        corner_coords = view_points(corners_3d, camera_intrinsic, True).T[:, :2].tolist()
    
        # Keep only corners that fall within the image.
        final_coords = post_process_coords(corner_coords, (1600, 900))
    
        return final_coords

    for scene_idx, scene in enumerate(nusc.scene):
        if scene_idx >= 425:
            break
        sample_tokens = nusc.field2token('sample', 'scene_token', scene['token'])
        for sample_token in sample_tokens:
            sample = nusc.get('sample', sample_token)
            camera_data = nusc.get('sample_data', sample['data'][C])
            calibrated_sensor = nusc.get('calibrated_sensor', camera_data['calibrated_sensor_token'])
            pose_rec = nusc.get('ego_pose', camera_data['ego_pose_token'])
            camera_intrinsic = np.array(calibrated_sensor['camera_intrinsic']) # Get the camera intrinsic matrix
            image_path = camera_data['filename']
            image_data = image_to_base64(image_path)

            annotations = []  # Collect annotations for each image
            for ann_token in sample['anns']:
                ann_record = nusc.get('sample_annotation', ann_token)
                #print(ann_record)
                if ann_record['category_name'] not in remove_categories:
                    box = Box(ann_record['translation'], ann_record['size'], Quaternion(ann_record['rotation']))
                    bbox_2d = convert_3d_to_2d_bbox(box, camera_intrinsic)
                    if bbox_2d:
                        annotations.append({
                            'category_name': ann_record['category_name'],
                            'bb_size': bbox_2d
                        })

            # Append the image with all annotations to the dataset
            if annotations:
                dataset.append({
                    'image_data': image_data,
                    'annotations': annotations,
                    #'sample_token': sample_token  # Optionally, you can store sample token for reference
                })

    # Replace categories in the dataset
    dataset = replace_categories(dataset, category_replacements)

    return dataset

Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 29.376 seconds.
Reverse indexing ...
Done reverse indexing in 6.0 seconds.


In [46]:
# Create your custom dataset
thesis_dataset1 = create_custom_dataset(nusc,'CAM_FRONT')
print("thesis_dataset1: done")
thesis_dataset2 = create_custom_dataset(nusc,'CAM_FRONT_RIGHT')
print("thesis_dataset2: done")
thesis_dataset3 = create_custom_dataset(nusc,'CAM_FRONT_LEFT')
print("thesis_dataset3: done")
thesis_dataset4 = create_custom_dataset(nusc,'CAM_BACK_RIGHT')
print("thesis_dataset4: done")
thesis_dataset5 = create_custom_dataset(nusc,'CAM_BACK_LEFT')
print("thesis_dataset5: done")
thesis_dataset6 = create_custom_dataset(nusc,'CAM_BACK')
print("thesis_dataset6: done")
thesis_dataset7 = create_custom_dataset(nusc,'CAM_BACK')
print("thesis_dataset7: done")
thesis_dataset8 = create_custom_dataset(nusc,'CAM_BACK_LEFT')
print("thesis_dataset8: done")
thesis_dataset = thesis_dataset1 + thesis_dataset2 + thesis_dataset3 + thesis_dataset4 + thesis_dataset5 + thesis_dataset6 + thesis_dataset7 + thesis_dataset8

thesis_dataset1: done
thesis_dataset2: done
thesis_dataset3: done
thesis_dataset4: done
thesis_dataset5: done
thesis_dataset6: done
thesis_dataset7: done
thesis_dataset8: done


In [82]:
len(thesis_dataset)

113740

In [48]:
def count_category_occurrences(dataset):
    """
    Count occurrences of each category in the dataset.
    """
    category_counts = {}

    # Iterate over the dataset and count occurrences of each category
    for data in dataset:
        annotations = data['annotations']
        for ann in annotations:
            category = ann['category_name']
            if category in category_counts:
                category_counts[category] += 1
            else:
                category_counts[category] = 1

    # Print the counts of each category
    print("Category Counts:")
    for category, count in category_counts.items():
        print(f"{category}: {count}")

In [50]:
count_category_occurrences(thesis_dataset)

Category Counts:
truck: 85138
pedestrian: 209943
barrier: 135615
car: 431135
motorcycle: 10723
bicycle: 12168
bus: 11873
trailer: 23900
animal: 537


In [51]:
import random

random.shuffle(thesis_dataset)

tset = thesis_dataset[:2999]

In [60]:
tset2 = thesis_dataset[3000:7000]

In [70]:
tset3 = thesis_dataset[7001:13001]

In [76]:
tset4 = thesis_dataset[13001:17001]

In [77]:
len(tset4)

4000

In [78]:
count_category_occurrences(tset4)

Category Counts:
pedestrian: 7294
barrier: 5044
truck: 2893
car: 14943
bus: 402
trailer: 851
motorcycle: 368
bicycle: 439
animal: 21


In [79]:
import os
import base64
from PIL import Image

def create_yolo_dataset(dataset):
    yolo_dataset = []

    class_id_map = {
        "truck": 0,
        "pedestrian": 1,
        "bus": 2,
        "car": 3,
        "barrier": 4,
        "trailer": 5,
        "motorcycle": 6,
        "bicycle": 7
    }

    # Define directory paths based on dataset type
    images_dir = f'yolon5/images/'
    labels_dir = f'yolon5/labels/'

    # Create the directories if they don't exist
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for idx, data in enumerate(dataset):
        image_data = data['image_data']
        image_path = os.path.join(images_dir, f'{idx}.jpg')

        with open(image_path, 'wb') as img_file:
            img_file.write(base64.b64decode(image_data))

        # Open the image and resize to [1200,600]
        image = Image.open(image_path)
        resized_image = image.resize((960, 480))
        resized_image.save(image_path)  # Save the resized image

        annotations = data['annotations']
        label_content = ''

        for ann in annotations:
            bbox = ann['bb_size']
            x_min, y_min, x_max, y_max = bbox

            # Calculate new bounding box coordinates based on the resized image
            width_ratio = 960 / 1600  # Ratio of new width to original width
            height_ratio = 480 / 900  # Ratio of new height to original height

            new_x_min = x_min * width_ratio
            new_y_min = y_min * height_ratio
            new_bbox_width = (x_max - x_min) * width_ratio
            new_bbox_height = (y_max - y_min) * height_ratio
            new_x_center = new_x_min + new_bbox_width / 2
            new_y_center = new_y_min + new_bbox_height / 2

            category_name = ann['category_name']
            if category_name != "animal":
                class_id = class_id_map.get(category_name, -1)  # Get class ID from the map, default to -1 if not found
    
                # Append bounding box annotation to label content
                label_content += f"{class_id} {new_x_center/960} {new_y_center/480} {new_bbox_width/960} {new_bbox_height/480}\n"

        # Save bounding box annotations to label file
        label_path = os.path.join(labels_dir, f'{idx}.txt')

        with open(label_path, 'w') as label_file:
            label_file.write(label_content)

        yolo_dataset.append({'image_path': image_path, 'label_path': label_path})

    return yolo_dataset

In [80]:
yolo= create_yolo_dataset(tset4)

In [ ]:
!zip -r yolon5.zip yolon5